# Week 7 & 8 Exercises
# Rahul Rajeev

In [102]:
# libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import regex as re

In [103]:
# reading the csv
candy_df = pd.read_excel('candyhierarchy2017.xlsx')

In [104]:
candy_df.head()

,Internal ID,Q1: GOING OUT?,Q2: GENDER,Q3: AGE,Q4: COUNTRY,"Q5: STATE, PROVINCE, COUNTY, ETC",Q6 | 100 Grand Bar,Q6 | Anonymous brown globs that come in black and orange wrappers\t(a.k.a. Mary Janes),Q6 | Any full-sized candy bar,Q6 | Black Jacks,...,Q8: DESPAIR OTHER,Q9: OTHER COMMENTS,Q10: DRESS,Unnamed: 113,Q11: DAY,Q12: MEDIA [Daily Dish],Q12: MEDIA [Science],Q12: MEDIA [ESPN],Q12: MEDIA [Yahoo],"Click Coordinates (x, y)"
0,90258773,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,90272821,No,Male,44,USA,NM,MEH,DESPAIR,JOY,MEH,...,NaN,Bottom line is Twix is really the only candy w...,White and gold,NaN,Sunday,NaN,1.0,NaN,NaN,"(84, 25)"
2,90272829,NaN,Male,49,USA,Virginia,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,90272840,No,Male,40,us,or,MEH,DESPAIR,JOY,MEH,...,NaN,Raisins can go to hell,White and gold,NaN,Sunday,NaN,1.0,NaN,NaN,"(75, 23)"
4,90272841,No,Male,23,usa,exton pa,JOY,DESPAIR,JOY,DESPAIR,...,NaN,NaN,White and gold,NaN,Friday,NaN,1.0,NaN,NaN,"(70, 10)"


## Chapter 7:

In [105]:
# 1. Removing duplicates
# check for no duplicates because each survey has a unique id
nodupes = candy_df.drop_duplicates()
print('Before:', candy_df.shape, 'After:', nodupes.shape)

Before: (2460, 120) After: (2460, 120)


In [106]:
# drop id column
nodupes = nodupes.drop(['Internal ID'], axis = 1)

In [107]:
# 2. dropping rows with all NaN elements (empty survey data)
clean = nodupes.dropna(how='all')
print('Before:', nodupes.shape, 'After:', clean.shape)

Before: (2460, 119) After: (2439, 119)


# Chapter 8:

In [108]:
# 3. pivot table
# converting the age column to numerical values
clean[['Q3: AGE']] = clean[['Q3: AGE']].apply(pd.to_numeric, errors='coerce')

# pivot table that compares gender to average age answered for specific dress
pivot_table = clean.pivot_table(index = 'Q2: GENDER', columns = 'Q10: DRESS', values = 'Q3: AGE', aggfunc = 'median')
pivot_table

C:\Users\rahul\anaconda3\lib\site-packages\pandas\core\frame.py:3069: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


Q10: DRESS,Blue and black,White and gold
Q2: GENDER,,
Female,38.0,41.0
I'd rather not say,41.0,45.0
Male,41.0,44.0
Other,30.0,34.5


In [109]:
# 4. reshape using stack(); unloading the dress columns into a separate column
pivot_table2 = pivot_table.stack()
pivot_table2

Q2: GENDER          Q10: DRESS    
Female              Blue and black    38.0
                    White and gold    41.0
I'd rather not say  Blue and black    41.0
                    White and gold    45.0
Male                Blue and black    41.0
                    White and gold    44.0
Other               Blue and black    30.0
                    White and gold    34.5
dtype: float64

## **Chapter 10:**

In [110]:
# selecting only the reviews about candy
candy = []
for col in clean.columns:
    if 'Q6' in col:
        candy.append(col)

In [111]:
onlycandy = clean.filter(candy)
onlycandy

,Q6 | 100 Grand Bar,Q6 | Anonymous brown globs that come in black and orange wrappers\t(a.k.a. Mary Janes),Q6 | Any full-sized candy bar,Q6 | Black Jacks,Q6 | Bonkers (the candy),Q6 | Bonkers (the board game),Q6 | Bottle Caps,Q6 | Box'o'Raisins,Q6 | Broken glow stick,Q6 | Butterfinger,...,Q6 | Three Musketeers,Q6 | Tolberone something or other,Q6 | Trail Mix,Q6 | Twix,"Q6 | Vials of pure high fructose corn syrup, for main-lining into your vein",Q6 | Vicodin,Q6 | Whatchamacallit Bars,Q6 | White Bread,Q6 | Whole Wheat anything,Q6 | York Peppermint Patties
1,MEH,DESPAIR,JOY,MEH,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,...,JOY,JOY,DESPAIR,JOY,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MEH,DESPAIR,JOY,MEH,MEH,DESPAIR,MEH,DESPAIR,DESPAIR,MEH,...,DESPAIR,JOY,MEH,JOY,DESPAIR,JOY,JOY,DESPAIR,DESPAIR,DESPAIR
4,JOY,DESPAIR,JOY,DESPAIR,MEH,DESPAIR,MEH,DESPAIR,DESPAIR,MEH,...,JOY,JOY,DESPAIR,JOY,MEH,JOY,JOY,DESPAIR,DESPAIR,JOY
5,JOY,DESPAIR,JOY,NaN,NaN,NaN,MEH,MEH,DESPAIR,JOY,...,JOY,JOY,MEH,JOY,DESPAIR,DESPAIR,JOY,DESPAIR,DESPAIR,JOY
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2455,JOY,DESPAIR,MEH,DESPAIR,DESPAIR,MEH,MEH,DESPAIR,DESPAIR,MEH,...,MEH,MEH,JOY,JOY,MEH,JOY,DESPAIR,MEH,DESPAIR,MEH
2456,MEH,DESPAIR,JOY,NaN,NaN,NaN,NaN,DESPAIR,DESPAIR,JOY,...,MEH,MEH,DESPAIR,JOY,NaN,NaN,JOY,DESPAIR,MEH,JOY
2457,MEH,DESPAIR,JOY,DESPAIR,MEH,JOY,DESPAIR,MEH,MEH,DESPAIR,...,JOY,JOY,MEH,MEH,MEH,JOY,MEH,DESPAIR,DESPAIR,MEH
2458,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [112]:
# stripping the extra characters to show individuality
cleaned = []
for col in onlycandy.columns:
    newcol = re.sub('[Q6|]','',col).rstrip().lstrip()
    cleaned.append(newcol)

In [113]:
# replacing the existing columns
onlycandy.columns = cleaned

In [115]:
# 5. crosstab of review frequency vs the candy, unfortunately, not all columns are shown
df1 = onlycandy.melt(var_name='columns', value_name='index')
pd.crosstab(index=df1['index'], columns=df1['columns'])

columns,100 Grand Bar,Abstained from M&M'ing.,Anonymous brown globs that come in black and orange wrappers\t(a.k.a. Mary Janes),Any full-sized candy bar,Black Jacks,Blue M&M's,Bonkers (the board game),Bonkers (the candy),Bottle Caps,Box'o'Raisins,...,Tic Tacs,Tolberone something or other,Trail Mix,Twix,"Vials of pure high fructose corn syrup, for main-lining into your vein",Vicodin,Whatchamacallit Bars,White Bread,Whole Wheat anything,York Peppermint Patties
index,,,,,,,,,,,,,,,,,,,,,
DESPAIR,85,693,1089,17,793,120,546,495,560,1179,...,754,84,836,71,1146,723,288,1455,1289,232
JOY,873,218,176,1559,92,1018,193,116,465,117,...,264,1320,228,1413,233,707,840,44,117,1105
MEH,755,607,461,212,617,595,715,857,670,476,...,728,350,688,286,289,241,509,204,307,418


In [116]:
# 6. Grouping with Dicts/Series: grouping by day and counting the number of entries per column
groupbyday = clean.groupby(by='Q11: DAY').count()
groupbyday

,Q1: GOING OUT?,Q2: GENDER,Q3: AGE,Q4: COUNTRY,"Q5: STATE, PROVINCE, COUNTY, ETC",Q6 | 100 Grand Bar,Q6 | Anonymous brown globs that come in black and orange wrappers\t(a.k.a. Mary Janes),Q6 | Any full-sized candy bar,Q6 | Black Jacks,Q6 | Bonkers (the candy),...,Q7: JOY OTHER,Q8: DESPAIR OTHER,Q9: OTHER COMMENTS,Q10: DRESS,Unnamed: 113,Q12: MEDIA [Daily Dish],Q12: MEDIA [Science],Q12: MEDIA [ESPN],Q12: MEDIA [Yahoo],"Click Coordinates (x, y)"
Q11: DAY,,,,,,,,,,,,,,,,,,,,,
Friday,1055,1085,1062,1082,1070,1038,1052,1086,924,906,...,548,441,242,1076,6,56,833,70,47,1000
Sunday,621,641,625,637,629,611,610,634,526,515,...,341,258,134,634,3,28,520,29,19,594


# Chapter 11:

In [117]:
# practicing date time conversions

# loading example: https://www.kaggle.com/datasets/mayurdalvi/date-time-handling
messages = pd.read_csv('messages.csv')
messages

,date,msg
0,2013-12-15 00:50:00,ищу на сегодня мужика 37
1,2014-04-29 23:40:00,ПАРЕНЬ БИ ИЩЕТ ДРУГА СЕЙЧАС!! СМС ММС 0955532826
2,2012-12-30 00:21:00,Днепр.м 43 позн.с д/ж *.о 067.16.34.576
3,2014-11-28 00:31:00,КИЕВ ИЩУ Д/Ж ДО 45 МНЕ СЕЙЧАС СКУЧНО 093 629 9...
4,2013-10-26 23:11:00,Зая я тебя никогда не обижу люблю тебя!) Даше
...,...,...
995,2012-03-16 00:50:00,ПАРЕНЬ СДЕЛАЕТ МАССАЖ ЖЕНЩИНАМ -066-877-32-44
996,2014-01-23 23:14:00,сельский п 23 ищу девушку для отношений
997,2012-10-15 23:37:00,Д+Д ДЛЯ серьезных отношений. Мой номер 093-156...
998,2012-06-21 23:34:00,7 ДНЕПР М.34 ПОЗ.С Д/Ж ДЛЯ ВСТРЕЧ.Т.098 809 15 14


In [118]:
# 7. Convert between string and date time
messages['Datetime']= pd.to_datetime(messages['date'])
messages

,date,msg,Datetime
0,2013-12-15 00:50:00,ищу на сегодня мужика 37,2013-12-15 00:50:00
1,2014-04-29 23:40:00,ПАРЕНЬ БИ ИЩЕТ ДРУГА СЕЙЧАС!! СМС ММС 0955532826,2014-04-29 23:40:00
2,2012-12-30 00:21:00,Днепр.м 43 позн.с д/ж *.о 067.16.34.576,2012-12-30 00:21:00
3,2014-11-28 00:31:00,КИЕВ ИЩУ Д/Ж ДО 45 МНЕ СЕЙЧАС СКУЧНО 093 629 9...,2014-11-28 00:31:00
4,2013-10-26 23:11:00,Зая я тебя никогда не обижу люблю тебя!) Даше,2013-10-26 23:11:00
...,...,...,...
995,2012-03-16 00:50:00,ПАРЕНЬ СДЕЛАЕТ МАССАЖ ЖЕНЩИНАМ -066-877-32-44,2012-03-16 00:50:00
996,2014-01-23 23:14:00,сельский п 23 ищу девушку для отношений,2014-01-23 23:14:00
997,2012-10-15 23:37:00,Д+Д ДЛЯ серьезных отношений. Мой номер 093-156...,2012-10-15 23:37:00
998,2012-06-21 23:34:00,7 ДНЕПР М.34 ПОЗ.С Д/Ж ДЛЯ ВСТРЕЧ.Т.098 809 15 14,2012-06-21 23:34:00


In [119]:
# 8. Generate date range
min(messages['Datetime'])

# separating the dates and times
messages['Dates'] = pd.to_datetime(messages['date']).dt.date
messages['Time'] = pd.to_datetime(messages['date']).dt.time

# creating date range from 2012 to 2013
samplerange = pd.date_range(start = min(messages['Dates']), end = '2013-01-01')

# dataframe of messages from 2012-2013
sampleset = messages[messages['Dates'].isin(samplerange)]
sampleset

,date,msg,Datetime,Dates,Time
85,2012-10-05 00:22:00,ПОЗНАКОМЛЮСЬ С СИМП ДЕВУШКОЙ Я СООТВЕТСТВУЮ.ОТ...,2012-10-05 00:22:00,2012-10-05,00:22:00
87,2012-03-21 00:54:00,подписчика,2012-03-21 00:54:00,2012-03-21,00:54:00
347,2012-03-21 01:42:00,Девушки вы где. давайте общаться через чат,2012-03-21 01:42:00,2012-03-21,01:42:00
439,2012-08-07 00:48:00,Познакомлюсь с приятной девушкой ! Сергей 22. ...,2012-08-07 00:48:00,2012-08-07,00:48:00
